In [29]:
# imports
import os
from skimage.measure import label,regionprops
from skimage.morphology import binary_opening, erosion, dilation, ball
from skimage.segmentation import watershed
from scipy.ndimage import median_filter as median
import numpy as np
import nrrd
from scripts.ImageSliceViewer3D import ImageSliceViewer3D as isv 

In [26]:
# lists of image/mask files
img_path = './data/images'
msk_path = './data/masks'

all_images = sorted([os.path.join(img_path,f) for f in os.listdir(img_path)])
all_masks = sorted([os.path.join(msk_path,f) for f in os.listdir(msk_path)])

In [39]:
# first pass of segmentation pipeline
def segPipeline(imgList,mskList):
    
    num_lesions = []
    all_IoU = []
    all_meanIoU = []
    
    # track the patient number
    patient_num = 1
    
    for i in range(len(imgList)):
        
        # read image and corresponding mask
        im_V,img_d = nrrd.read(imgList[i])
        msk_V,msk_d = nrrd.read(mskList[i])
        
        # if the image and mask do not have the same size, do not process
        if not (img_d['sizes']==msk_d['sizes']).all():
            print('ERROR, {}: image and mask must be of the same size'.format(imgList[i]))
            continue
            
        # label the mask into connected regions
        lesion_labels,num_lesions = label(msk_V,return_num=True)
        lesion_props = regionprops(lesion_labels)
        
        # preprocess the original image:
        #        (i)   filter to minimize noise
        #        (ii)  obtain the morphological gradient
        denoised = median(im_V,3)
        gradient_image = dilation(denoised,ball(2)) - erosion(denoised,ball(2))
        
        # work towards a whole-image segmentation
        #        (i)   threshold the morphological gradient
        #        (ii)  label the threshold image
        #        (iii) apply watershed to threshold image using labels as markers
        threshold_gradient = gradient_image < 350 
        markers = label(threshold_gradient)
        labels = watershed(gradient_image, markers)
        
        # track the lesion number
        lesion_count = 1
        
        for obj in nodule_props:
            
            # isolate the centroid of the nodule -- this will be our lesion identifier for automation
            centroid = [round(i) for i in obj.centroid] 
            print('Patient #{}; Lesion #{}; Centroid: {}'.format(patient_num,lesion_count,centroid))
            
            # the segmentation
            water_mask = labels == labels[centroid[0],centroid[1],centroid[2]]

            
            
            lesion_count += 1
        
        
        patient_num += 1
        

# run the pipeline
segPipeline(all_images,all_masks)


# iou_mask = np.count_nonzero(np.logical_and(water_mask,msk_V))/np.count_nonzero(np.logical_or(water_mask,msk_V))
# iou_back = np.count_nonzero(np.logical_and(~water_mask,~msk_V))/np.count_nonzero(np.logical_or(~water_mask,~msk_V))

# iou_mask 

# iou_back

# (iou_mask + iou_back) / 2    # <-- mean IoU

Patient #1; Lesion #1; Centroid: [367, 316, 90]
Patient #2; Lesion #1; Centroid: [360, 345, 184]
Patient #3; Lesion #1; Centroid: [198, 308, 83]
Patient #3; Lesion #2; Centroid: [219, 221, 84]
Patient #3; Lesion #3; Centroid: [347, 369, 66]
Patient #3; Lesion #4; Centroid: [350, 367, 75]
Patient #4; Lesion #1; Centroid: [322, 142, 76]
Patient #5; Lesion #1; Centroid: [377, 407, 88]
Patient #5; Lesion #2; Centroid: [398, 127, 77]
Patient #5; Lesion #3; Centroid: [407, 312, 80]
Patient #6; Lesion #1; Centroid: [333, 392, 68]
Patient #6; Lesion #2; Centroid: [340, 191, 54]
Patient #6; Lesion #3; Centroid: [354, 363, 70]
Patient #6; Lesion #4; Centroid: [357, 371, 84]
Patient #7; Lesion #1; Centroid: [266, 293, 125]
Patient #7; Lesion #2; Centroid: [290, 193, 108]
Patient #7; Lesion #3; Centroid: [278, 212, 102]
Patient #8; Lesion #1; Centroid: [270, 369, 88]
Patient #8; Lesion #2; Centroid: [290, 123, 71]
Patient #9; Lesion #1; Centroid: [324, 304, 167]
Patient #9; Lesion #2; Centroid: [3

In [40]:
regionprops?